#Import Statements

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.preprocessing import LabelEncoder

#Reading the Metadata

In [2]:
root = "/content/drive/MyDrive/Capstone Project/Complete Dataset/"

In [3]:
metadata = pd.read_csv(root+"HAM10000_metadata")

In [4]:
metadata['dx'].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [5]:
le = LabelEncoder()

In [6]:
le.fit(metadata['dx'])

LabelEncoder()

In [7]:
metadata['class_label']=le.transform(metadata['dx'])

In [8]:
metadata['class_label'].value_counts()

5    6705
4    1113
2    1099
1     514
0     327
6     142
3     115
Name: class_label, dtype: int64

#Organising The Dataset

In [9]:
# classes=metadata['dx'].unique().tolist()

In [10]:
# import shutil

In [11]:
# source_dir="/content/drive/MyDrive/Capstone Project/Complete Dataset/HAM10000_all_images/"
# dest_dir="/content/drive/MyDrive/Capstone Project/Complete Dataset/Class Wise Organised Images/"

In [12]:
# class_images=[]
# for i in tqdm(classes,):
#   if not os.path.exists(dest_dir + str(i) + "/"): 
#     os.mkdir(dest_dir + str(i) + "/")
#   image_ids = metadata[metadata['dx'] == i]['image_id']
#   class_images.extend(image_ids)
#   for id in tqdm(class_images):
#     if os.path.exists(dest_dir + i + "/"+id+".jpg"):
#       continue
#     else:
#       shutil.copyfile((source_dir + "/"+ id +".jpg"), (dest_dir + i + "/"+id+".jpg"))
#   class_images=[] 

In [13]:
# from PIL import Image as PImage
# def loadImages(root_path):
#   imagesList_1 = os.listdir(root_path + "HAM10000_images_part_1")
#   imagesList_2 = os.listdir(root_path + "HAM10000_images_part_2")
#   loadedImages = []
#   for image in imagesList_1:
#       img = PImage.open(root_path + "HAM10000_images_part_1" + "/" + image)
#       loadedImages.append(img)
#   for image in imagesList_2:
#       img = PImage.open(root_path + "HAM10000_images_part_2" + "/" + image)
#       loadedImages.append(img)
#   return loadedImages

#Build A Model

##Preprocessing

In [31]:
train_datagen = ImageDataGenerator(rescale= 1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [32]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [33]:
train_data = train_datagen.flow_from_directory("/content/drive/MyDrive/Capstone Project/Complete Dataset/Test_Val_Train Split/train",target_size=(32,32),batch_size=16,class_mode='categorical')

Found 8010 images belonging to 7 classes.


In [34]:
test_data = test_datagen.flow_from_directory("/content/drive/MyDrive/Capstone Project/Complete Dataset/Test_Val_Train Split/test",target_size=(32,32),batch_size=32,class_mode='categorical')

Found 1007 images belonging to 7 classes.


##Building The CNN

In [96]:
cnn = tf.keras.models.Sequential()

In [97]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[32, 32, 3]))

In [98]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [99]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [100]:
cnn.add(tf.keras.layers.Flatten())

In [101]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [102]:
cnn.add(tf.keras.layers.Dense(units=7, activation='sigmoid'))

In [103]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'],)

In [104]:
cnn.fit(x=train_data,validation_data=test_data,epochs = 25)

Epoch 1/25
501/501 [==============================] - 3455s 7s/step - loss: 0.9735 - accuracy: 0.6688 - val_loss: 0.9178 - val_accuracy: 0.6703
Epoch 2/25
501/501 [==============================] - 131s 261ms/step - loss: 0.8909 - accuracy: 0.6792 - val_loss: 0.8520 - val_accuracy: 0.6852
Epoch 3/25
501/501 [==============================] - 115s 229ms/step - loss: 0.8488 - accuracy: 0.6909 - val_loss: 0.8651 - val_accuracy: 0.6783
Epoch 4/25
501/501 [==============================] - 115s 229ms/step - loss: 0.8070 - accuracy: 0.7021 - val_loss: 0.7914 - val_accuracy: 0.6951
Epoch 5/25
501/501 [==============================] - 110s 219ms/step - loss: 0.7692 - accuracy: 0.7161 - val_loss: 0.7624 - val_accuracy: 0.7140
Epoch 6/25
501/501 [==============================] - 116s 231ms/step - loss: 0.7333 - accuracy: 0.7248 - val_loss: 0.7312 - val_accuracy: 0.7269
Epoch 7/25
501/501 [==============================] - 120s 240ms/step - loss: 0.7279 - accuracy: 0.7272 - val_loss: 0.7892 - v

In [105]:
cnn.save('/content/drive/MyDrive/Capstone Project/Saved Models/first_model.h5')